# AI-Legal-Summarizer

Problem Description: Develop an AI-powered system to analyze and summarize legal documents, court judgments, and laws, making them more accessible to citizens. The system should extract key details like dates, parties, and case outcomes while handling complex legal jargon and retaining essential information.

## Loading Legal Documents

In [1]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [2]:
# Load Legal Documents
doc_paths = ["court_case.pdf", "constitution.pdf"]
docs = []

for doc_file in doc_paths:
    if doc_file.endswith(".pdf"):
        loader = PyPDFLoader(doc_file)
    elif doc_file.endswith(".docx"):
        loader = Docx2txtLoader(doc_file)
    else:
        continue

    docs.extend(loader.load())

In [3]:
# Split Text into Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
document_chunks = text_splitter.split_documents(docs)

# Store in Vector Database
vector_db = Chroma.from_documents(document_chunks, OpenAIEmbeddings())

## Creating a Context-Aware Retreival Chain

In [6]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Retrieval Chain
def _get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()

    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Based on the query, retrieve the most relevant legal information, including case dates, parties, and judgments."),
    ])
    return create_history_aware_retriever(llm, retriever, prompt)

## Summarization Chain

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage

llm = ChatOpenAI(model="gpt-4o", temperature=0.2)

def get_legal_summary_chain(llm):
    retriever_chain = _get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         """You are an AI assistant specializing in summarizing legal documents.
         Your task is to extract key details such as case title, court name, judgment date,
         parties involved, legal provisions, arguments, and final verdict.
         
         You will use the provided context, but if the context is not sufficient, rely on your general knowledge.
         
         Context:
         {context}"""),  
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])

    summarization_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, summarization_chain)


In [12]:
conversation_rag_chain = get_legal_summary_chain(llm)

messages = [
    {"role": "user", "content": "Summarize the case 'XYZ vs ABC'."}
]

messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

response_message = "*(Legal Summary)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

Based on the provided context, the case titled "Child in Conflict with Law through his Mother vs. State of Karnataka and Another" was heard in the Supreme Court of India, with the judgment delivered by Justices C.T. Ravikumar and Rajesh Bindal. The case involved an appeal by a Child in Conflict with Law (CCL) against an order from the High Court, which had set aside a decision by the Juvenile Justice Board.

Key details include:

- **Judgment Date**: May 7, 2024
- **Parties Involved**: 
  - Appellant: Child in Conflict with Law through his Mother
  - Respondents: State of Karnataka and Another
- **Legal Provisions**: The case involved sections 376(i), 342 of the Indian Penal Code (IPC), and sections 4, 5, 6, 7, and 8 of the Protection of Children from Sexual Offences Act, 2012. It also referenced Section 7(4) of the Juvenile Justice Act.
- **Arguments**: The main issue was whether the CCL should be tried by the Juvenile Justice Board or as an adult by the Children's Court. The Principa